# Final term project

## 1. Members
* 2017572006 SooYeon Lee
* 2017572012 YunSong Jeong

## 2. Environments
* Python 3.5

## 3. Backend
* tensorflow-cpu

## 3. Preprocessing

In [1]:
import os
import numpy as np
# import cPickle
import _pickle as cPickle
import matplotlib.pylab as plt
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.ticker as mtick
## No bold!!

matplotlib.font_manager._rebuild()

import matplotlib as mpl
fmt = '{x:,.0f}'
tick = mtick.StrMethodFormatter(fmt)
mpl.rc('font',family='serif',serif='Times New Roman', size=15)

In [2]:
from scipy.stats import kurtosis
from scipy.stats import skew

In [3]:
filepath = 'D:/DEAP_DONOTDISTRIBUTE/'
files = os.listdir(filepath)
print (len(files), files[0])

32 s01.dat


In [3]:
with open(filepath+files[0], 'rb') as f:
    d = cPickle.load(f, encoding='latin1') 

In [4]:
d['data'].shape

(40, 40, 8064)

1. Every team needs to try and reproduce the results from Tripathi et al. (Using Deep and Convolutional Neural Networks for Accurate Emotion Classification on DEAP Dataset) on the EEG data ONLY. This means to take the 8064 EEG values per trial and channel (which they confusingly call "experiment" in their paper...) and to extract the mean, median, maximum, minimum, standard deviation, variance, range, skewness, and kurtosis in 10 non-overlapping "batches" of ~807 values each. Since these are 9 values for each "batch" and you have 10 "batches", this gives you 90 values for each trial and each channel. You then add the the same 9 values evaluated for the WHOLE 8064 EEG values, yielding 99 values. For some reason, Tripathi et al. add the trial number and the participant number as well, giving 101 values. For the first part of this experiment, do this as well. 
You now have for each participant 40 * (101 * 40) numbers. Tripathi et al. throw the (101*40) values into a standard CNN with 2D convolutions, effectively treating this like a "picture". Use the MNIST template as a guidance on how to set up this problem with a CNN and play around with some layers / filters / optimizers until you find a nice network that trains ok and also gives "good" results on the test set [yes, I know - here we would have really like to have a validation set!!]. Report the results of the best THREE networks in your notebook and include their code for both the TWO-CLASS and the THREE-CLASS results. 
Also use another classifier [like a decision tree] as a benchmark and include its result on both two-class and three-class results. If you want, this classifier can, for example, treat the data for each trial and participant like a 1600 measurements x 101 feature dimensions problem.
Train your best network and the benchmark classifier also on 99 values only and compare the results. Does including the trial number and participant number make a difference?? To get statistical certainty on your interpretations, use a Mann-Whitney-U test comparing the 32 values of two methods. Comment on this in your notebook.

1. for each participant and each channel and per trials, 99 values evaluated. (-> $31 * 40 * 40 * 101$)
2. Using $31 * 40 * 40 * 101$ , make model and test with 1 partipant.
3. use CNN (or another methods).

In [106]:
# from sklearn.preprocessing import normalize

all_result = []
labels = []
for par_num, file in enumerate(files):
    print ("For participants : %s"%(par_num))
    with open(filepath+file, 'rb') as f:
        d = cPickle.load(f, encoding='latin1')
        for i in range(0,40):
            # Each trial (i)
            trial_result = []
            result = dict()
        
            if d['labels'][i][0]<5:
                label2=[1,0]
            else:
                label2=[0,1]
            
            if d['labels'][i][0]<4:
                label3=[1,0,0]
            elif d['labels'][i][0]<6:
                label3 = [0,1,0]
            else:
                label3 = [0,0,1]
                
            result["label2"] = label2
            result["label3"] = label3
            
            for j in range(0,40):
                
                #Each channel
                row = []
                seq = d['data'][i][j] # / np.linalg.norm(d['data'][i][j])
                row.append(np.mean(seq))
                row.append(np.median(seq))
                row.append(np.max(seq))
                row.append(np.min(seq))
                row.append(np.std(seq))
                row.append(np.var(seq))
                row.append(np.ptp(seq))
                row.append(skew(seq))
                row.append(kurtosis(seq))
                
                for k in range(0,8060,int(8064/10)):
                    if k==7254:
                        val = seq[k:]
                    else:
                        val = seq[k:k+int(8064/10)]
                    row.append(np.mean(val))
                    row.append(np.median(val))
                    row.append(np.max(val))
                    row.append(np.min(val))
                    row.append(np.std(val))
                    row.append(np.var(val))
                    row.append(np.ptp(val))
                    row.append(skew(val))
                    row.append(kurtosis(val))
                row.append(par_num)
                row.append(i)
                trial_result.append(row)
        
            result["value"] = trial_result
            result["trial"] = i
            result["participant"] = par_num
            all_result.append(result)

For participants : 0
For participants : 1
For participants : 2
For participants : 3
For participants : 4
For participants : 5
For participants : 6
For participants : 7
For participants : 8
For participants : 9
For participants : 10
For participants : 11
For participants : 12
For participants : 13
For participants : 14
For participants : 15
For participants : 16
For participants : 17
For participants : 18
For participants : 19
For participants : 20
For participants : 21
For participants : 22
For participants : 23
For participants : 24
For participants : 25
For participants : 26
For participants : 27
For participants : 28
For participants : 29
For participants : 30
For participants : 31


In [107]:
import pandas as pd
data = pd.DataFrame(all_result)
data.head()

,label2,label3,participant,trial,value
0,"[0, 1]","[0, 0, 1]",0,0,"[[-0.028258582130643233, -0.00962694430445854,..."
1,"[0, 1]","[0, 0, 1]",0,1,"[[-0.08271212687175221, -0.00845419199872266, ..."
2,"[0, 1]","[0, 0, 1]",0,2,"[[0.06068887303620551, 0.0916055125512342, 20...."
3,"[1, 0]","[0, 1, 0]",0,3,"[[-0.027772601671089484, -0.03865440270217024,..."
4,"[0, 1]","[0, 0, 1]",0,4,"[[-0.06253660164932238, -0.0829994184864522, 1..."


In [108]:
data.shape

(1280, 5)

## make test set, train set

In [198]:
for par_num in range(1,32):
    X_train = np.array(list(data[(data["participant"]!=par_num)]["value"].values))
    X_train = X_train.reshape(X_train.shape[0],1,40,101).astype('float32')
    Y_train = np.array(list(data[(data["participant"]!=par_num)]["label2"].values))
    
    X_test = np.array(list(data[data["participant"]==par_num]["value"].values))
    X_test = X_test.reshape(X_test.shape[0],1,40,101).astype('float32')
    Y_test = np.array(list(data[data["participant"]==par_num]["label2"].values))
    
    break

In [161]:
X_train.shape

(1240, 1, 40, 101)

In [4]:
from keras.models import Sequential
from keras.layers import Input
from keras.models import Model
from keras.layers import Flatten
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, pooling
from keras.utils import np_utils
import keras
from keras.utils import plot_model

C:\Users\ahroobe\Anaconda2\envs\ahroobe\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [308]:
num_classes = Y_test.shape[1]

def cnn_model():
    # create model
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(1, 40, 101), activation='softmax',data_format='channels_first'))
    model.add(pooling.MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(128, (12, 12) ,activation='relu', data_format='channels_first'))
    model.add(pooling.MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))
    
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


In [309]:
model = cnn_model()
model.fit(X_train, Y_train,  epochs=10, batch_size=20, verbose=1)

Epoch 1/10
1240/1240 [==============================] - 41s 33ms/step - loss: 0.7287 - acc: 0.5177
Epoch 2/10
1240/1240 [==============================] - 42s 34ms/step - loss: 0.6859 - acc: 0.5637
Epoch 3/10
1240/1240 [==============================] - 40s 32ms/step - loss: 0.6797 - acc: 0.5605
Epoch 4/10
1240/1240 [==============================] - 41s 33ms/step - loss: 0.6689 - acc: 0.6032
Epoch 5/10
1240/1240 [==============================] - 41s 33ms/step - loss: 0.6221 - acc: 0.6573
Epoch 6/10
1240/1240 [==============================] - 40s 32ms/step - loss: 0.5143 - acc: 0.7661
Epoch 7/10
1240/1240 [==============================] - 40s 32ms/step - loss: 0.3528 - acc: 0.8460
Epoch 8/10
1240/1240 [==============================] - 40s 32ms/step - loss: 0.2560 - acc: 0.8960
Epoch 9/10
1240/1240 [==============================] - 41s 33ms/step - loss: 0.1683 - acc: 0.9363
Epoch 10/10
1240/1240 [==============================] - 41s 33ms/step - loss: 0.0935 - acc: 0.9653


In [310]:
scores = model.evaluate(X_test, Y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 60.00%


In [311]:
y_pred = model.predict(X_test)
y_pred

array([[2.92892396e-01, 7.07107544e-01],
       [1.22251533e-01, 8.77748489e-01],
       [1.53063685e-02, 9.84693646e-01],
       [8.22472036e-01, 1.77527949e-01],
       [5.00543237e-01, 4.99456793e-01],
       [8.46627772e-01, 1.53372258e-01],
       [1.45581380e-01, 8.54418576e-01],
       [2.26321012e-01, 7.73678958e-01],
       [4.80971575e-01, 5.19028366e-01],
       [1.78794742e-01, 8.21205318e-01],
       [7.34078363e-02, 9.26592231e-01],
       [8.64493489e-01, 1.35506511e-01],
       [9.80882049e-01, 1.91179290e-02],
       [9.87327278e-01, 1.26727335e-02],
       [2.32042298e-02, 9.76795793e-01],
       [9.80702102e-01, 1.92978438e-02],
       [6.50447071e-01, 3.49552870e-01],
       [2.56730765e-01, 7.43269265e-01],
       [5.47088720e-02, 9.45291102e-01],
       [9.23167825e-01, 7.68322200e-02],
       [8.60536158e-01, 1.39463797e-01],
       [6.27990246e-01, 3.72009784e-01],
       [6.48531765e-02, 9.35146809e-01],
       [2.27124663e-03, 9.97728765e-01],
       [4.019344

Trash

In [219]:
y_train = []
for lab in Y_train:
    lab = list(lab)
    if lab==[0,1]:
        y_train.append("HIGH")
    else:
        y_train.append("LOW")

In [245]:
y_test = []
for lab in Y_test:
    lab = list(lab)
    if lab==[0,1]:
        y_test.append("HIGH")
    else:
        y_test.append("LOW")

In [226]:
x_train = X_train.reshape(1240,4040)
x_test = X_test.reshape(40,4040)

In [253]:
from sklearn import tree
from sklearn.metrics import accuracy_score

clf = tree.DecisionTreeClassifier(max_depth=20)
clf = clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print ("Accuracy :  %f" % accuracy_score(y_test, list(y_pred)))

Accuracy :  0.600000


In [256]:
from sklearn.metrics import confusion_matrix
cnf_matrix = confusion_matrix(y_test, y_pred)
cnf_matrix

array([[20,  5],
       [11,  4]], dtype=int64)

# Experiment 2. 
The second experiment will now try to add other features. 
Here, I let you free reign on HOW to do this. You can follow the same logic and use mean, … for the other 8 “channels” (EOG, EDA, …) and in this manner increase the number of “channels” in the “picture" - this would be an example of “early, feature-level fusion”. Note that if you do this, you need to worry about normalization of the EEG versus the other channels - neural networks do not like different scales or ranges. Hence, if you do this, remember to do some sort of normalization on the different modalities first!!!
You can also train different networks or classifiers on the other channels and then use the final outputs in decision-level fusion.
Report the results of the best-performing network and/or classifier on your multi-modal fusion for both two-class and three-class classification schemes. Can you beat the results of the first experiment? To get statistical certainty, use a Mann-Whitney-U test comparing the 32 values of your best algorithm from experiment 1 and the best from experiment 2. Are they significantly different?


In [5]:
from sklearn.preprocessing import normalize
valence = [0,2,12,15,19,36]
all_result = []
labels = []
for par_num, file in enumerate(files):
    print ("For participants : %s"%(par_num))
    with open(filepath+file, 'rb') as f:
        d = cPickle.load(f, encoding='latin1')
        for i in range(0,40):
            # Each trial (i)
            trial_result = []
            result = dict()
        
            if d['labels'][i][0]<5:
                label2=[1,0]
            else:
                label2=[0,1]
            
            if d['labels'][i][0]<4:
                label3=[1,0,0]
            elif d['labels'][i][0]<6:
                label3 = [0,1,0]
            else:
                label3 = [0,0,1]
                
            result["label2"] = label2
            result["label3"] = label3
            
            for j in valence:
                
              
                seq = list(d['data'][i][j]/np.linalg.norm(d['data'][i][j]))
                
                trial_result.append(seq)
        
            result["value"] = trial_result
            result["trial"] = i
            result["participant"] = par_num
            all_result.append(result)


For participants : 0
For participants : 1
For participants : 2
For participants : 3
For participants : 4
For participants : 5
For participants : 6
For participants : 7
For participants : 8
For participants : 9
For participants : 10
For participants : 11
For participants : 12
For participants : 13
For participants : 14
For participants : 15
For participants : 16
For participants : 17
For participants : 18
For participants : 19
For participants : 20
For participants : 21
For participants : 22
For participants : 23
For participants : 24
For participants : 25
For participants : 26
For participants : 27
For participants : 28
For participants : 29
For participants : 30
For participants : 31


In [6]:
import pandas as pd
data = pd.DataFrame(all_result)
data.head()

,label2,label3,participant,trial,value
0,"[0, 1]","[0, 0, 1]",0,0,"[[0.0027148305210395124, 0.004733574395258349,..."
1,"[0, 1]","[0, 0, 1]",0,1,"[[0.026809379557441247, 0.03343392086065311, 0..."
2,"[0, 1]","[0, 0, 1]",0,2,"[[0.002612810988497244, -0.002754104114280221,..."
3,"[1, 0]","[0, 1, 0]",0,3,"[[-0.02152534311230555, -0.00918403879225411, ..."
4,"[0, 1]","[0, 0, 1]",0,4,"[[-0.00572421505918025, -0.015119992104360965,..."


In [23]:
temp = np.array(list(data[(data["participant"]!=0)]["value"].values))
temp.reshape(1240,1,128,378).astype('float32').shape

(1240, 1, 128, 378)

In [24]:
temp.shape

(1240, 6, 8064)

In [7]:
for par_num in range(1,32):
    X_train = np.array(list(data[(data["participant"]!=par_num)]["value"].values))
    X_train = X_train.reshape(X_train.shape[0],1,128,378).astype('float32')
    Y_train = np.array(list(data[(data["participant"]!=par_num)]["label2"].values))
    
    X_test = np.array(list(data[data["participant"]==par_num]["value"].values))
    X_test = X_test.reshape(X_test.shape[0],1,128,378).astype('float32')
    Y_test = np.array(list(data[data["participant"]==par_num]["label2"].values))
    
    break

In [8]:
num_classes = Y_test.shape[1]

def cnn_model():
    # create model
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(1, 128, 378), activation='softmax',data_format='channels_first'))
    model.add(pooling.MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(128, (12, 12) ,activation='relu', data_format='channels_first'))
    model.add(pooling.MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))
    
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


In [9]:
model = cnn_model()
model.fit(X_train, Y_train,  epochs=5, batch_size=20, verbose=1)

Epoch 1/5
1240/1240 [==============================] - 812s 655ms/step - loss: 0.7357 - acc: 0.5573
Epoch 2/5
1240/1240 [==============================] - 815s 657ms/step - loss: 0.6870 - acc: 0.5637
Epoch 3/5
1240/1240 [==============================] - 806s 650ms/step - loss: 0.6855 - acc: 0.5637
Epoch 4/5
1240/1240 [==============================] - 807s 651ms/step - loss: 0.6854 - acc: 0.5637
Epoch 5/5
 140/1240 [==>...........................] - ETA: 11:42 - loss: 0.6850 - acc: 0.5643

KeyboardInterrupt: 

In [10]:
scores = model.evaluate(X_test, Y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 62.50%


In [11]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.layers import Dropout

In [15]:
for par_num in range(1,32):
    X_train = np.array(list(data[(data["participant"]!=par_num)]["value"].values))
    X_train = X_train.transpose(0,2,1).astype('float32')
    
    Y_train = np.array(list(data[(data["participant"]!=par_num)]["label2"].values))
    
    X_test = np.array(list(data[data["participant"]==par_num]["value"].values))
    X_test = X_test.transpose(0,2,1).astype('float32')
    Y_test = np.array(list(data[data["participant"]==par_num]["label2"].values))
    break

In [24]:
neurons = 20
model = Sequential()
model.add(LSTM(neurons, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(neurons, activation='tanh'))
model.add(Dense(2,activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_9 (LSTM)                (None, 8064, 20)          2160      
_________________________________________________________________
lstm_10 (LSTM)               (None, 20)                3280      
_________________________________________________________________
dense_7 (Dense)              (None, 2)                 42        
Total params: 5,482
Trainable params: 5,482
Non-trainable params: 0
_________________________________________________________________
None


In [25]:
model.fit(X_train, Y_train, epochs=3)

Epoch 1/3
1240/1240 [==============================] - 287s 232ms/step - loss: 0.6898 - acc: 0.5411
Epoch 2/3
1240/1240 [==============================] - 291s 235ms/step - loss: 0.6853 - acc: 0.5637
Epoch 3/3


KeyboardInterrupt: 

no deep learning

# 0.5637

In [38]:
from sklearn.preprocessing import normalize

all_result = []
labels = []
for par_num, file in enumerate(files):
    print ("For participants : %s"%(par_num))
    with open(filepath+file, 'rb') as f:
        d = cPickle.load(f, encoding='latin1')
        for i in range(0,40):
            # Each trial (i)
            trial_result = []
            result = dict()
        
            if d['labels'][i][0]<5:
                label2=[1,0]
            else:
                label2=[0,1]
            
            if d['labels'][i][0]<4:
                label3=[1,0,0]
            elif d['labels'][i][0]<6:
                label3 = [0,1,0]
            else:
                label3 = [0,0,1]
                
            result["label2"] = label2
            result["label3"] = label3
            
            for j in range(0,40):
                
                #Each channel
                row = []
                seq = d['data'][i][j] - d['data'][i][j][0] # / np.linalg.norm(d['data'][i][j])
                row.append(np.mean(seq))
                row.append(np.median(seq))
                row.append(np.max(seq))
                row.append(np.min(seq))
                row.append(np.std(seq))
                row.append(np.var(seq))
                row.append(np.ptp(seq))
                row.append(skew(seq))
                row.append(kurtosis(seq))
                
                for k in range(0,8060,int(8064/10)):
                    if k==7254:
                        val = seq[k:]
                    else:
                        val = seq[k:k+int(8064/10)]
                    row.append(np.mean(val))
                    row.append(np.median(val))
                    row.append(np.max(val))
                    row.append(np.min(val))
                    row.append(np.std(val))
                    row.append(np.var(val))
                    row.append(np.ptp(val))
                    row.append(skew(val))
                    row.append(kurtosis(val))
                trial_result.append(row)
        
            result["value"] = trial_result
            result["trial"] = i
            result["participant"] = par_num
            all_result.append(result)

For participants : 0
For participants : 1
For participants : 2
For participants : 3
For participants : 4
For participants : 5
For participants : 6
For participants : 7
For participants : 8
For participants : 9
For participants : 10
For participants : 11
For participants : 12
For participants : 13
For participants : 14
For participants : 15
For participants : 16
For participants : 17
For participants : 18
For participants : 19
For participants : 20
For participants : 21
For participants : 22
For participants : 23
For participants : 24
For participants : 25
For participants : 26
For participants : 27
For participants : 28
For participants : 29
For participants : 30
For participants : 31


In [39]:
import pandas as pd
data = pd.DataFrame(all_result)
data.head()

,label2,label3,participant,trial,value
0,"[0, 1]","[0, 0, 1]",0,0,"[[-0.9764902631258351, -0.9578586252996505, 15..."
1,"[0, 1]","[0, 0, 1]",0,1,"[[-10.3428871767865, -10.26862924191347, 7.235..."
2,"[0, 1]","[0, 0, 1]",0,2,"[[-0.9523606846263067, -0.9214440451112781, 19..."
3,"[1, 0]","[0, 1, 0]",0,3,"[[7.630655822844306, 7.6197740218132255, 24.96..."
4,"[0, 1]","[0, 0, 1]",0,4,"[[1.7485713212436582, 1.7281085044065283, 17.3..."


In [42]:
accuracy = []
for par_num in range(0,31):
    print ("Testing %s participants"%par_num)
    X_train = np.array(list(data[(data["participant"]!=par_num)]["value"].values))
    X_train = X_train.reshape(X_train.shape[0],1,40,99).astype('float32')
    Y_train = np.array(list(data[(data["participant"]!=par_num)]["label2"].values))
    
    X_test = np.array(list(data[data["participant"]==par_num]["value"].values))
    X_test = X_test.reshape(X_test.shape[0],1,40,99).astype('float32')
    Y_test = np.array(list(data[data["participant"]==par_num]["label2"].values))
    model = cnn_model()
    model.fit(X_train, Y_train,  epochs=10, batch_size=20, verbose=1)
    
    scores = model.evaluate(X_test, Y_test, verbose=0)
    print("Accuracy for %s : %.2f%%" % (par_num, scores[1]*100))
    accuracy.append(scores[1]*100)

Testing 0 participants
Epoch 1/10
1240/1240 [==============================] - 41s 33ms/step - loss: 0.7028 - acc: 0.5403
Epoch 2/10
1240/1240 [==============================] - 39s 32ms/step - loss: 0.6690 - acc: 0.5968
Epoch 3/10
1240/1240 [==============================] - 39s 31ms/step - loss: 0.6136 - acc: 0.6694
Epoch 4/10
1240/1240 [==============================] - 39s 32ms/step - loss: 0.5176 - acc: 0.7411
Epoch 5/10
1240/1240 [==============================] - 39s 32ms/step - loss: 0.3643 - acc: 0.8452
Epoch 6/10
1240/1240 [==============================] - 39s 32ms/step - loss: 0.2356 - acc: 0.9073
Epoch 7/10
1240/1240 [==============================] - 39s 32ms/step - loss: 0.1083 - acc: 0.9637
Epoch 8/10
1240/1240 [==============================] - 42s 34ms/step - loss: 0.0922 - acc: 0.9726
Epoch 9/10
1240/1240 [==============================] - 39s 32ms/step - loss: 0.0882 - acc: 0.9742
Epoch 10/10
1240/1240 [==============================] - 40s 32ms/step - loss: 0.1140 

1240/1240 [==============================] - 39s 31ms/step - loss: 0.0955 - acc: 0.9677
Accuracy for 7 : 50.00%
Testing 8 participants
Epoch 1/10
1240/1240 [==============================] - 41s 33ms/step - loss: 0.7106 - acc: 0.5694
Epoch 2/10
1240/1240 [==============================] - 39s 32ms/step - loss: 0.6710 - acc: 0.6016
Epoch 3/10
1240/1240 [==============================] - 39s 31ms/step - loss: 0.6327 - acc: 0.6758
Epoch 4/10
1240/1240 [==============================] - 39s 32ms/step - loss: 0.5233 - acc: 0.7395
Epoch 5/10
1240/1240 [==============================] - 40s 32ms/step - loss: 0.4044 - acc: 0.8234
Epoch 6/10
1240/1240 [==============================] - 39s 32ms/step - loss: 0.2590 - acc: 0.8976
Epoch 7/10
1240/1240 [==============================] - 39s 31ms/step - loss: 0.1505 - acc: 0.9435
Epoch 8/10
1240/1240 [==============================] - 38s 31ms/step - loss: 0.1511 - acc: 0.9492
Epoch 9/10
1240/1240 [==============================] - 39s 31ms/step - l

1240/1240 [==============================] - 39s 31ms/step - loss: 0.1618 - acc: 0.9411
Epoch 9/10
1240/1240 [==============================] - 39s 31ms/step - loss: 0.1327 - acc: 0.9492
Epoch 10/10
1240/1240 [==============================] - 39s 31ms/step - loss: 0.1712 - acc: 0.9403
Accuracy for 15 : 42.50%
Testing 16 participants
Epoch 1/10
1240/1240 [==============================] - 43s 34ms/step - loss: 0.7159 - acc: 0.5339
Epoch 2/10
1240/1240 [==============================] - 41s 33ms/step - loss: 0.6863 - acc: 0.5750
Epoch 3/10
1240/1240 [==============================] - 41s 33ms/step - loss: 0.6755 - acc: 0.5798
Epoch 4/10
1240/1240 [==============================] - 40s 32ms/step - loss: 0.6508 - acc: 0.6194
Epoch 5/10
1240/1240 [==============================] - 39s 32ms/step - loss: 0.6086 - acc: 0.6500
Epoch 6/10
1240/1240 [==============================] - 39s 31ms/step - loss: 0.4737 - acc: 0.7702
Epoch 7/10
1240/1240 [==============================] - 39s 31ms/step 

1240/1240 [==============================] - 39s 31ms/step - loss: 0.2842 - acc: 0.8839
Epoch 7/10
1240/1240 [==============================] - 39s 31ms/step - loss: 0.2060 - acc: 0.9194
Epoch 8/10
1240/1240 [==============================] - 39s 31ms/step - loss: 0.1854 - acc: 0.9315
Epoch 9/10
1240/1240 [==============================] - 39s 31ms/step - loss: 0.1142 - acc: 0.9605
Epoch 10/10
1240/1240 [==============================] - 39s 32ms/step - loss: 0.0884 - acc: 0.9710
Accuracy for 23 : 47.50%
Testing 24 participants
Epoch 1/10
1240/1240 [==============================] - 44s 36ms/step - loss: 0.7135 - acc: 0.5419
Epoch 2/10
1240/1240 [==============================] - 44s 35ms/step - loss: 0.6787 - acc: 0.5661
Epoch 3/10
1240/1240 [==============================] - 42s 34ms/step - loss: 0.6598 - acc: 0.6032
Epoch 4/10
1240/1240 [==============================] - 40s 32ms/step - loss: 0.6160 - acc: 0.6613
Epoch 5/10
1240/1240 [==============================] - 40s 33ms/step 

## Average Pooling

In [51]:
poolingsses = Y_test.shape[1]

def cnn_model2():
    # create model
    model = Sequential()
    model.add(Conv2D(128, (12, 12), input_shape=(1, 36, 101), activation='softmax',data_format='channels_first'))
    model.add(pooling.AveragePooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))
    
#     model.add(Conv2D(128, (12, 12) ,activation='relu', data_format='channels_first'))
#     model.add(pooling.AveragePooling2D(pool_size=(2,2)))
#     model.add(Dropout(0.2))
    
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# from sklearn.preprocessing import normalize

# all_result = []
# labels = []
# for par_num, file in enumerate(files):
#     print ("For participants : %s"%(par_num))
#     with open(filepath+file, 'rb') as f:
#         d = cPickle.load(f, encoding='latin1')
#         for i in range(0,40):
#             # Each trial (i)
#             trial_result = []
#             result = dict()
        
#             if d['labels'][i][0]<5:
#                 label2=[1,0]
#             else:
#                 label2=[0,1]
            
#             if d['labels'][i][0]<4:
#                 label3=[1,0,0]
#             elif d['labels'][i][0]<6:
#                 label3 = [0,1,0]
#             else:
#                 label3 = [0,0,1]
                
#             result["label2"] = label2
#             result["label3"] = label3
            
#             for j in range(0,36):
                
#                 #Each channel
#                 row = []
               
#                 row.append(np.mean(seq))
#                 row.append(np.median(seq))
#                 row.append(np.max(seq))
#                 row.append(np.min(seq))
#                 row.append(np.std(seq))
#                 row.append(np.var(seq))
#                 row.append(np.ptp(seq))
#                 row.append(skew(seq))
#                 row.append(kurtosis(seq))
                
#                 for k in range(0,8060,int(8064/10)):
#                     if k==7254:
#                         val = seq[k:]
#                     else:
#                         val = seq[k:k+int(8064/10)]
#                     row.append(np.mean(val))
#                     row.append(np.median(val))
#                     row.append(np.max(val))
#                     row.append(np.min(val))
#                     row.append(np.std(val))
#                     row.append(np.var(val))
#                     row.append(np.ptp(val))
#                     row.append(skew(val))
#                     row.append(kurtosis(val))
#                 row.append(par_num)
#                 row.append(i)
#                 trial_result.append(row)
        
#             result["value"] = trial_result
#             result["trial"] = i
#             result["participant"] = par_num
#             all_result.append(result)
            
data2 = pd.DataFrame(all_result)   
accuracy2 = []
for par_num in range(0,32):
    print ("Testing %s participants"%par_num)
    X_train = np.array(list(data2[(data2["participant"]!=par_num)]["value"].values))
    X_train = X_train.reshape(X_train.shape[0],1,36,101).astype('float32')
    Y_train = np.array(list(data2[(data2["participant"]!=par_num)]["label2"].values))
    
    X_test = np.array(list(data2[data2["participant"]==par_num]["value"].values))
    X_test = X_test.reshape(X_test.shape[0],1,36,101).astype('float32')
    Y_test = np.array(list(data2[data2["participant"]==par_num]["label2"].values))
    
    model2 = cnn_model2()
    model2.fit(X_train, Y_train,  epochs=10, batch_size=20, verbose=1)
    scores = model2.evaluate(X_test, Y_test, verbose=0)
    print("Accuracy for %s : %.2f%%" % (par_num, scores[1]*100))
    accuracy2.append(scores[1]*100)

Testing 0 participants
Epoch 1/10
1240/1240 [==============================] - 58s 47ms/step - loss: 0.7152 - acc: 0.5460
Epoch 2/10
1240/1240 [==============================] - 54s 43ms/step - loss: 0.6993 - acc: 0.5387
Epoch 3/10
1240/1240 [==============================] - 52s 42ms/step - loss: 0.6890 - acc: 0.5685
Epoch 4/10
1240/1240 [==============================] - 52s 42ms/step - loss: 0.6858 - acc: 0.5685
Epoch 5/10
1240/1240 [==============================] - 51s 41ms/step - loss: 0.6841 - acc: 0.5685
Epoch 6/10
1240/1240 [==============================] - 53s 43ms/step - loss: 0.6858 - acc: 0.5685
Epoch 7/10
1240/1240 [==============================] - 54s 44ms/step - loss: 0.6864 - acc: 0.5685
Epoch 8/10
1240/1240 [==============================] - 52s 42ms/step - loss: 0.6880 - acc: 0.5685
Epoch 9/10
1240/1240 [==============================] - 52s 42ms/step - loss: 0.6844 - acc: 0.5685
Epoch 10/10
1240/1240 [==============================] - 51s 41ms/step - loss: 0.6853 

1240/1240 [==============================] - 51s 41ms/step - loss: 0.6851 - acc: 0.5661
Accuracy for 7 : 55.00%
Testing 8 participants
Epoch 1/10
1240/1240 [==============================] - 54s 44ms/step - loss: 0.7604 - acc: 0.5282
Epoch 2/10
1240/1240 [==============================] - 51s 41ms/step - loss: 0.7194 - acc: 0.5065
Epoch 3/10
1240/1240 [==============================] - 53s 42ms/step - loss: 0.7187 - acc: 0.5040
Epoch 4/10
1240/1240 [==============================] - 52s 42ms/step - loss: 0.7103 - acc: 0.5210
Epoch 5/10
1240/1240 [==============================] - 51s 41ms/step - loss: 0.6886 - acc: 0.5444
Epoch 6/10
1240/1240 [==============================] - 51s 41ms/step - loss: 0.6894 - acc: 0.5645
Epoch 7/10
1240/1240 [==============================] - 51s 41ms/step - loss: 0.6848 - acc: 0.5669
Epoch 8/10
1240/1240 [==============================] - 51s 41ms/step - loss: 0.6862 - acc: 0.5669
Epoch 9/10
1240/1240 [==============================] - 53s 43ms/step - l

1240/1240 [==============================] - 53s 42ms/step - loss: 0.6840 - acc: 0.5718
Epoch 9/10
1240/1240 [==============================] - 52s 42ms/step - loss: 0.6846 - acc: 0.5718
Epoch 10/10
1240/1240 [==============================] - 57s 46ms/step - loss: 0.6846 - acc: 0.5718
Accuracy for 15 : 37.50%
Testing 16 participants
Epoch 1/10
1240/1240 [==============================] - 57s 46ms/step - loss: 0.7496 - acc: 0.5403
Epoch 2/10
1240/1240 [==============================] - 52s 42ms/step - loss: 0.6895 - acc: 0.5508
Epoch 3/10
1240/1240 [==============================] - 53s 43ms/step - loss: 0.6890 - acc: 0.5468
Epoch 4/10
1240/1240 [==============================] - 56s 45ms/step - loss: 0.6921 - acc: 0.5403
Epoch 5/10
1240/1240 [==============================] - 55s 44ms/step - loss: 0.6873 - acc: 0.5661
Epoch 6/10
1240/1240 [==============================] - 54s 43ms/step - loss: 0.6856 - acc: 0.5661
Epoch 7/10
1240/1240 [==============================] - 53s 43ms/step 

1240/1240 [==============================] - 52s 42ms/step - loss: 0.6937 - acc: 0.5371
Epoch 7/10
1240/1240 [==============================] - 53s 43ms/step - loss: 0.6916 - acc: 0.5605
Epoch 8/10
1240/1240 [==============================] - 52s 42ms/step - loss: 0.6913 - acc: 0.5532
Epoch 9/10
1240/1240 [==============================] - 52s 42ms/step - loss: 0.6912 - acc: 0.5427
Epoch 10/10
1240/1240 [==============================] - 52s 42ms/step - loss: 0.6925 - acc: 0.5532
Accuracy for 23 : 47.50%
Testing 24 participants
Epoch 1/10
1240/1240 [==============================] - 59s 47ms/step - loss: 0.7175 - acc: 0.5355
Epoch 2/10
1240/1240 [==============================] - 52s 42ms/step - loss: 0.6960 - acc: 0.5169
Epoch 3/10
1240/1240 [==============================] - 52s 42ms/step - loss: 0.6900 - acc: 0.5669
Epoch 4/10
1240/1240 [==============================] - 52s 42ms/step - loss: 0.6843 - acc: 0.5669
Epoch 5/10
1240/1240 [==============================] - 51s 41ms/step 

1240/1240 [==============================] - 52s 42ms/step - loss: 0.7141 - acc: 0.5008
Epoch 5/10
1240/1240 [==============================] - 52s 42ms/step - loss: 0.7204 - acc: 0.5226
Epoch 6/10
1240/1240 [==============================] - 54s 43ms/step - loss: 0.7007 - acc: 0.5452
Epoch 7/10
1240/1240 [==============================] - 55s 44ms/step - loss: 0.6923 - acc: 0.5379
Epoch 8/10
1240/1240 [==============================] - 54s 43ms/step - loss: 0.6896 - acc: 0.5677
Epoch 9/10
1240/1240 [==============================] - 53s 43ms/step - loss: 0.6876 - acc: 0.5677
Epoch 10/10
1240/1240 [==============================] - 52s 42ms/step - loss: 0.6864 - acc: 0.5677
Accuracy for 31 : 50.00%


In [31]:
num_classes = Y_test.shape[1]

def cnn_model():
    # create model
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(1, 40, 99), activation='softmax',data_format='channels_first'))
    model.add(pooling.MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(128, (12, 12) ,activation='relu', data_format='channels_first'))
    model.add(pooling.MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))
    
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = cnn_model()
model.fit(X_train, Y_train,  epochs=5, batch_size=20, verbose=1)

Epoch 1/5
1240/1240 [==============================] - 40s 32ms/step - loss: 0.7110 - acc: 0.5323
Epoch 2/5
1240/1240 [==============================] - 41s 33ms/step - loss: 0.6702 - acc: 0.5895
Epoch 3/5
1240/1240 [==============================] - 39s 32ms/step - loss: 0.6446 - acc: 0.6306
Epoch 4/5
1240/1240 [==============================] - 40s 32ms/step - loss: 0.5660 - acc: 0.6968
Epoch 5/5
1240/1240 [==============================] - 39s 32ms/step - loss: 0.4271 - acc: 0.7968


In [32]:
scores = model.evaluate(X_test, Y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 60.00%
